# Map Dates

In [2]:
import pandas as pd

In [ ]:
with pd.read_json("../data/quotebank-"+str(2019)+".json.bz2", lines=True,  chunksize = 10000, compression = 'bz2') as df_reader:
    for chunk in df_reader: #we read chunk by chunk in order not to store everything in memory
        for index, row in chunk.iterrows():
            for el in row.quotations:
                if el["quoteID"] == "2019-02-10-028331":
                    print("WE ARE FUCKED")

In [49]:
def filter_dataframe(year):
# This function will be used to filter the dataset selecting the quotes of a given year containing a given word
# in a free text search fashion.

    list_df = []
    with pd.read_json("../data/quotebank-"+year+".json.bz2", lines=True,  chunksize = 10000, compression = 'bz2') as df_reader:
        import concurrent.futures
        executor = concurrent.futures.ThreadPoolExecutor(30)
        for chunk in df_reader: #we read chunk by chunk in order not to store everything in memory
            executor.submit(filter_chunk, list_df, chunk)
        executor.shutdown()
            
    df = pd.concat(list_df) #we concatenate the dataframes together to obtain a unique one
    return df

In [128]:
def filter_chunk(list_df, chunk):
    chunk.drop(columns=["articleID", "phase", "title", "url", "articleLength", "names"], inplace=True)
    chunk.quotations = chunk.quotations.transform(lambda x: filtering_func(x))
    chunk = chunk[chunk["quotations"].str.len() > 0]
    list_df.append(chunk)  #we append it to the list of dataframes
            
        
def filtering_func(lst):
    final_lst = []
    for el in lst:
        if (("Trump" in el["quotation"]) or ("Clinton" in el["quotation"])):
            final_lst.append(el["quoteID"])
    return final_lst
    

In [129]:
quoteID_date = filter_dataframe("2019")

KeyboardInterrupt: 

In [ ]:
quoteID_date

In [10]:
quoteID_date.to_pickle('2019_quoteID_date.pkl')

NameError: name 'quoteID_date' is not defined

In [3]:
quoteID_cleaned = {str(2000 + i) : pd.read_pickle('../data/' + str(2000 + i) + '_quoteID_date.pkl') for i in range(15,21)}
quoteID_cleaned

{'2015':                                                  quotations  \
 147       [2015-08-05-025201, 2015-08-05-055692, 2015-08...   
 185                  [2015-08-06-081642, 2015-08-06-061444]   
 243                  [2015-08-07-047067, 2015-08-07-046176]   
 257                                     [2015-08-07-100597]   
 667       [2015-08-19-099364, 2015-08-19-121637, 2015-08...   
 ...                                                     ...   
 10144325                                [2015-05-12-050394]   
 10144709                                [2015-07-15-021355]   
 10144886                                [2015-07-23-075404]   
 10144944                                [2015-07-27-056746]   
 10144971                                [2015-07-28-102216]   
 
                         date  
 147      2015-08-05 22:33:14  
 185      2015-08-06 17:46:00  
 243      2015-08-07 17:20:35  
 257      2015-08-07 20:46:40  
 667      2015-08-19 09:49:00  
 ...                      ... 

In [4]:
clinton_cleaned = pd.read_pickle('../data/df_Clinton_cleaned.pkl')
clinton_cleaned

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
26,2015-10-25-000242,"' It is not now, nor has it ever been, the gol...",Bernie Sanders,[Q359442],2015-10-25 14:12:35,1,"[[Bernie Sanders, 0.5395], [None, 0.3128], [Hi...",[http://examiner.com/article/bernie-sanders-sl...,E
215,2015-07-30-032957,"I am supporting Hillary Clinton for president,",None,[],2015-07-30 04:08:03,1,"[[None, 0.5004], [Miro Weinberger, 0.4696], [J...",[http://www.sevendaysvt.com/OffMessage/archive...,E
888,2015-09-16-006359,And I'm just pointing out the absurd on both s...,Kathleen Madigan,[Q6376814],2015-09-16 05:44:37,1,"[[Kathleen Madigan, 0.8025], [None, 0.1975]]",[http://northjersey.com/arts-and-entertainment...,E
2324,2015-08-15-008350,Clinton's College Hypocrisy Tour Rolls On,None,[],2015-08-15 23:24:29,1,"[[None, 0.8041], [Marco Rubio, 0.1016], [Hilla...",[http://miamiherald.typepad.com/nakedpolitics/...,E
2809,2015-08-25-044501,If Vice President Joe Biden decides to jump in...,None,[],2015-08-25 12:05:31,2,"[[None, 0.6475], [Joe Biden, 0.29], [Ryan Grim...",[http://feeds.huffingtonpost.com/c/35496/f/677...,E
...,...,...,...,...,...,...,...,...,...
5232184,2020-02-08-016054,"I had done a lot of presidential debates, but ...",Chris Wallace,"[Q21256789, Q2964884, Q2964885, Q37624906, Q51...",2020-02-08 04:40:01,1,"[[Chris Wallace, 0.9267], [None, 0.0702], [Hil...",[https://www.thewrap.com/fox-news-chris-wallac...,E
5235860,2020-01-06-061256,The main difference between Lindsey and his De...,David Woodard,[Q1177254],2020-01-06 12:00:30,6,"[[David Woodard, 0.7544], [None, 0.1797], [Lin...",[http://chron.com/entertainment/article/How-Li...,E
5235869,2020-04-09-052373,The model of Obama asking Bush and Clinton to ...,Bill Haslam,[Q862186],2020-04-09 23:04:21,1,"[[Bill Haslam, 0.905], [None, 0.0837], [Barack...",[http://www.nytimes.com/2020/04/09/us/politics...,E
5237423,2020-01-22-037250,I think the fact is that Mitch keeps telling y...,Patrick Leahy,[Q59315],2020-01-22 02:53:15,10,"[[Patrick Leahy, 0.8213], [None, 0.148], [Roge...",[http://wicz.com/story/41593610/inside-the-rep...,E


In [5]:
def filter_chunk_by_number_of_qids(initial_chunk, qid_number):
    return initial_chunk[initial_chunk["qids"].str.len() == qid_number]

In [7]:
clinton_cleaned= filter_chunk_by_number_of_qids(clinton_cleaned, 1)

clinton_cleaned

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
26,2015-10-25-000242,"' It is not now, nor has it ever been, the gol...",Bernie Sanders,[Q359442],2015-10-25 14:12:35,1,"[[Bernie Sanders, 0.5395], [None, 0.3128], [Hi...",[http://examiner.com/article/bernie-sanders-sl...,E
888,2015-09-16-006359,And I'm just pointing out the absurd on both s...,Kathleen Madigan,[Q6376814],2015-09-16 05:44:37,1,"[[Kathleen Madigan, 0.8025], [None, 0.1975]]",[http://northjersey.com/arts-and-entertainment...,E
6930,2015-10-22-051493,If [ Democratic presidential candidate former ...,Marco Rubio,[Q324546],2015-10-22 20:04:16,1,"[[Marco Rubio, 0.93], [None, 0.0685], [Hillary...",[http://breitbart.com/video/2015/10/22/rubio-i...,E
7374,2015-12-31-032035,I'm electable. I was elected in a purple state...,Jeb Bush,[Q221997],2015-12-31 03:29:00,7,"[[Jeb Bush, 0.8392], [None, 0.0925], [Hillary ...",[http://www.postandcourier.com/article/2015123...,E
9855,2015-11-12-104266,The incentive to invent episodes of discrimina...,Glenn Reynolds,[Q4392664],2015-11-12 21:40:00,2,"[[Glenn Reynolds, 0.3454], [Ed Driscoll, 0.322...","[http://pjmedia.com/instapundit/218734/, http:...",E
...,...,...,...,...,...,...,...,...,...
5222109,2020-03-06-025712,"I think that would have been a mistake, becaus...",Jennifer Palmieri,[Q18209402],2020-03-06 14:38:07,1,"[[Jennifer Palmieri, 0.9117], [None, 0.056], [...",[https://www.rollingstone.com/politics/politic...,E
5231803,2020-01-18-006266,"Chief Justice Rehnquist, when he presided over...",Dick Durbin,[Q434804],2020-01-18 05:01:08,1,"[[Dick Durbin, 0.898], [None, 0.079], [Charlie...",[https://www.washingtonexaminer.com/news/impea...,E
5235860,2020-01-06-061256,The main difference between Lindsey and his De...,David Woodard,[Q1177254],2020-01-06 12:00:30,6,"[[David Woodard, 0.7544], [None, 0.1797], [Lin...",[http://chron.com/entertainment/article/How-Li...,E
5235869,2020-04-09-052373,The model of Obama asking Bush and Clinton to ...,Bill Haslam,[Q862186],2020-04-09 23:04:21,1,"[[Bill Haslam, 0.905], [None, 0.0837], [Barack...",[http://www.nytimes.com/2020/04/09/us/politics...,E


In [8]:
def years_to_try(year):
    year_lst = [str(year)]
    for i in range(2020 - year):
        year_lst.append(str(year + i + 1))
    return year_lst

In [9]:
def update_quotes(quotes, quoteID_cleaned):
    new_quotes = quotes.copy()
    new_quotes = new_quotes.drop(columns=["date"])
    new_quotes["date"] = "no_date"
    count = 0
    for index, row in quotes.iterrows():
        if row.numOccurrences > 1:
            quoteID = row.quoteID
            earliest_date = row.date
            occurrences_lst = []
            for year in years_to_try(earliest_date.year):
                dataframe = quoteID_cleaned[year]
                occurrences = dataframe[dataframe["quotations"].transform(lambda x: quoteID in x)].date.to_list()
                occurrences_lst += occurrences
                if len(occurrences_lst) == row.numOccurrences:
                    break
            new_quotes.at[index, "date"] = occurrences_lst
        else:
            new_quotes.at[index, "date"] = [row.date]
        count += 1
        if (count % 100 == 0):
            print(count)
    return new_quotes

In [11]:
import time

start = time.time()

merged_quotes = update_quotes(clinton_cleaned[60000:], quoteID_cleaned)

end = time.time()
delta = end - start

print(delta)

merged_quotes

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
35.69540762901306


,quoteID,quotation,speaker,qids,numOccurrences,probas,urls,phase,date
17956726,2019-12-10-013563,Clinton and Pharrell Williams were on their fe...,Bernie Sanders,[Q359442],1,"[[Bernie Sanders, 0.3809], [Pharrell Williams,...",[https://www.chicagoreader.com/chicago/bernie-...,E,[2019-12-10 14:22:00]
17957341,2019-06-20-024004,"From the land of Funk are Mary J. Blige, Georg...",Anderson . Paak,[Q20810369],1,"[[Anderson . Paak, 0.5354], [Walt Dohrn, 0.284...",[http://www.blastr.com/syfywire/angry-birds-2-...,E,[2019-06-20 17:25:00]
17959871,2019-01-21-040434,It reminded me of the social media tactics tha...,Kristyn Wong-Tam,[Q6438493],1,"[[Kristyn Wong-Tam, 0.9138], [None, 0.0862]]",[https://www.thestar.com/opinion/contributors/...,E,[2019-01-21 13:07:34]
17964235,2019-10-30-112921,undo Hillary Clinton's failed legacy.,Tulsi Gabbard,[Q32620],32,"[[Tulsi Gabbard, 0.8967], [None, 0.1024], [Hil...",[http://uspolitics.einnews.com/article/5007150...,E,"[2019-10-30 12:45:56, 2019-10-30 12:45:56, 201..."
17967020,2019-07-22-007695,"But the questions should be asked, why were al...",President Trump,[Q22686],1,"[[President Trump, 0.4689], [None, 0.4552], [B...",[http://www.foxnews.com/media/media-drumbeat-f...,E,[2019-07-22 00:00:00]
...,...,...,...,...,...,...,...,...,...
5222109,2020-03-06-025712,"I think that would have been a mistake, becaus...",Jennifer Palmieri,[Q18209402],1,"[[Jennifer Palmieri, 0.9117], [None, 0.056], [...",[https://www.rollingstone.com/politics/politic...,E,[2020-03-06 14:38:07]
5231803,2020-01-18-006266,"Chief Justice Rehnquist, when he presided over...",Dick Durbin,[Q434804],1,"[[Dick Durbin, 0.898], [None, 0.079], [Charlie...",[https://www.washingtonexaminer.com/news/impea...,E,[2020-01-18 05:01:08]
5235860,2020-01-06-061256,The main difference between Lindsey and his De...,David Woodard,[Q1177254],6,"[[David Woodard, 0.7544], [None, 0.1797], [Lin...",[http://chron.com/entertainment/article/How-Li...,E,[]
5235869,2020-04-09-052373,The model of Obama asking Bush and Clinton to ...,Bill Haslam,[Q862186],1,"[[Bill Haslam, 0.905], [None, 0.0837], [Barack...",[http://www.nytimes.com/2020/04/09/us/politics...,E,[2020-04-09 23:04:21]


In [12]:
merged_quotes.to_pickle('../data/Clinton7_with_dates.pkl')

In [13]:
merged_quotes[(merged_quotes["date"].str.len() == 0) & (merged_quotes["quoteID"] > "2019")]

,quoteID,quotation,speaker,qids,numOccurrences,probas,urls,phase,date
18262716,2019-10-08-109006,were confident Mr. Weiner's actions would not ...,Anthony Weiner,[Q557810],2,"[[Anthony Weiner, 0.7146], [None, 0.1777], [Hi...",[http://feeds.businessinsider.com.au/~/6075732...,E,[]
18532122,2019-07-27-008979,Hillary Clinton was at my wedding. We had Henr...,Andrea Catsimatidis,[Q22639329],2,"[[Andrea Catsimatidis, 0.4199], [John Catsimat...",[https://www.businessinsider.in/Meet-the-self-...,E,[]
19695552,2019-01-29-083722,The Clinton administration was willing to allo...,George W. Bush,[Q207],39,"[[George W. Bush, 0.6487], [None, 0.2894], [Jo...",[http://watoday.com.au/world/north-america/rus...,E,[]
20119319,2019-11-25-059710,One wonders why there aren't a few more report...,Prince Andrew,[Q153330],2,"[[Prince Andrew, 0.6469], [None, 0.3402], [Hil...",[http://www.thetrumpet.com/21531-prince-andrew...,E,[]
20371857,2019-04-12-034509,I immediately suspected this was payback for t...,Trina Vargo,[Q7842294],2,"[[Trina Vargo, 0.6635], [None, 0.2209], [Bill ...",[https://www.washingtonexaminer.com/politics/r...,E,[]
20399983,2019-06-04-117087,What she learned was that impeachment basicall...,Tony Coelho,[Q2442506],2,"[[Tony Coelho, 0.7412], [None, 0.2042], [Ro Kh...","[http://hypervocal.com/items/2714368, http://t...",E,[]
20834843,2019-07-05-077564,Well [ West is ] concerned that I am inciting ...,Joe Maddon,[Q1093629],6,"[[Joe Maddon, 0.496], [None, 0.412], [Clint Hu...",[http://espn.go.com/mlb/story/_/id/27121710/pi...,E,[]
21417152,2019-10-08-097466,"Two, we see Verizon.Blackberry.net email metad...",James Comey,[Q167607],2,"[[James Comey, 0.86], [None, 0.1116], [Hillary...",[http://feeds.businessinsider.com.au/~/6075732...,E,[]
80893,2020-01-22-063878,"nothing like the Clinton rules,",Chuck Schumer,[Q380900],12,"[[Chuck Schumer, 0.7839], [None, 0.1613], [Mit...",[http://feeds.abcnews.com/Politics/trump-impea...,E,[]
141555,2020-02-19-006290,apparently the FBI... didn't treat [ Clinton's...,Sean Hannity,[Q557730],2,"[[Sean Hannity, 0.7625], [None, 0.2211], [Jame...",[https://mediamatters.org/fox-news/fox-news-le...,E,[]


In [19]:
quoteID_cleaned["2019"][quoteID_cleaned["2019"]["quotations"].transform(lambda x: "2019-02-04-014299" in x)]


,quotations,date
4820985,"[2019-02-04-014299, 2019-02-04-017176]",2019-02-04 14:15:00
11044255,"[2019-02-04-017176, 2019-02-04-014299]",2019-02-04 14:15:00


### Merge cleaned Trump/Clinton files

In [83]:
def merge_func(author_name,file_numbers):
    df_list=[]
    for number in file_numbers:
        df_chunk = pd.read_pickle('../data/' + author_name + str(number) + '_with_dates.pkl')
        df_list.append(df_chunk)
    df = pd.concat(df_list)
    return df

We merge every Clinton and Trump files into a single dataframe

In [84]:
Clinton_merged = merge_func("Clinton",[1,2,3,4,5,6,7])
Trump_merged = merge_func("Trump",[1,2,3])

### Dealing with missing values

We notice some missing dates after we clean and merge every files. This happens because certain quotes are missing in the article files. In order to deal with this missing values, we will fill them with the containing date in quoteID

In [85]:
def grab_date_from_quoteID(author_name, df):
    missing_values = df[(df["date"].str.len() == 0)]
    
    df.loc[df["date"].str.len() == 0, "date"] = pd.to_datetime(df[(df["date"].str.len() == 0)]["quoteID"].str.slice(stop=10))
    return df


In [86]:
Clinton_with_dates = grab_date_from_quoteID("Clinton", Clinton_merged)
Trump_with_dates = grab_date_from_quoteID("Trump", Trump_merged)

Save dataframes of Clinton and Trump

In [87]:
Clinton_with_dates.to_pickle('../data/Clinton_with_dates.pkl')
Trump_with_dates.to_pickle('../data/Trump_with_dates.pkl')